In [1]:
import FedUtils
from model.utils.config import cfg

### Arguments

In [2]:
class Arguments():
    def __init__(self):
        self.class_agnostic =False
        self.epochs = 3
        self.lr = 0.001
        self.lr_decay_step = 4
        self.dataset = 'KAIST'
        self.net = 'vgg16'
        self.batch_size = 24
        self.nw = 2
        self.cuda = True
        #self.round = 10
        self.mGPUs=True
        self.optimizer ="sgd"
        self.k=5
        #self.device = th.device("cpu")
        
args = Arguments()

### load model

In [7]:
round_i = 2
parties = 3
sub_folder = 'FedPer'

model_path = ['models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_campus_'+str(round_i)+'_3_718.pth',
'models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_road_'+str(round_i)+'_3_513.pth',
'models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_downtown_'+str(round_i)+'_3_566.pth']


imdb_classes =  ('__background__',  # always index 0
                          'person',
                          'people','cyclist'
                         )

model_list=[None] * parties
for i in range(parties):
    model_list[i],optimizer, start_round = FedUtils.load_model(imdb_classes,model_path[i], args,cfg)

Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth


In [8]:
def FedPer(model_list,ratio_list,mGPUs):
    
    model_tmp=[None] * parties
    #optims_tmp=[None] * parties

    for idx, my_model in enumerate(model_list):
        if mGPUs:
            my_model = my_model.module
            
        model_tmp[idx] = my_model.RCNN_base.state_dict()


    for key in model_tmp[0]:    
        #print(key)
        model_avg = 0

        for idx, model_tmp_content in enumerate(model_tmp):     # add each model              
            model_avg += ratio_list[idx] * model_tmp_content[key]
            
        for i in range(len(model_tmp)):  #copy to each model            
            model_tmp[i][key] = model_avg
    #copy back to original model
    for i in range(len(model_list)):  
        if mGPUs:
            model_list[i].module.RCNN_base.load_state_dict(model_tmp[i])
        else:
            model_list[i].RCNN_base.load_state_dict(model_tmp[i])
    return model_list

### load test image

In [6]:
from frcnn_helper import *


        
def getWeight(test_images,model_list, args):
    
    wk_list = []
    for fasterRCNN in model_list:
        if args.mGPUs:
            fasterRCNN = fasterRCNN.module
        X = get_features(fasterRCNN, test_images, args.batch_size)/255.0
        wk_value = within_cluster_dispersion(X, n_cluster=args.k)
        wk_list.append(wk_value)
        print(wk_value)
    
    return wk_list

In [9]:
import pickle
testimg_pickle_path = 'testimg2252.pkl'
with open(testimg_pickle_path, 'rb') as handle:
    test_images = pickle.load(handle)

In [10]:
round_i

2

In [ ]:
wk_list_curr = getWeight(test_images,model_list, args)

In [ ]:
wk_list_curr =[1.9260449632344736,
1.6288783338524226,
1.623319350129662]

In [13]:
from scipy.special import softmax
       
wk_list_prev=[2.210802457989739,
2.159582836258389,
2.469066878769875]

if round_i==1:
    wk_diff = wk_list_curr
else:
    wk_diff=[]
    for list1_c, list2_p in zip(wk_list_prev, wk_list_curr):        
        wk_diff.append(list1_c-list2_p)

print('diff={}'.format(wk_diff))

wk_ratio = softmax(wk_diff).tolist()    
print('wk_ratio={}'.format(wk_ratio))
wk_list_prev = wk_list_curr

diff=[0.28475749475526557, 0.5307045024059664, 0.8457475286402127]
wk_ratio=[0.24806273010321633, 0.31723050850525814, 0.4347067613915256]


In [14]:
model_list =FedPer(model_list,wk_ratio,args.mGPUs)

In [15]:
output_dir = "models/" + args.net + "/" + args.dataset + "/" + sub_folder
output_dir

'models/vgg16/KAIST/FedPer'

In [16]:
import os
from model.utils.net_utils import save_checkpoint
save_name = os.path.join(output_dir, 'faster_rcnn_KAIST_AVG_{}.pth'.format(round_i))
save_checkpoint({
  #'session': 1,
  'round': round_i,
  'model':  model_list[0].module.state_dict() if args.mGPUs else model_list[0].state_dict(), 
  'optimizer': optimizer.state_dict(),
  'pooling_mode': cfg.POOLING_MODE,
  'class_agnostic': args.class_agnostic,
}, save_name)

# backup

In [6]:
wk_ratio = [0.33333,0.33333,0.33333]   

model_list = FedUtils.avgWeight(model_list,wk_ratio)


In [14]:
ratio_list = [0.33333,0.33333,0.33333]   

model_list = FedPer(model_list,ratio_list,True)

In [10]:
model_list[0].load_state_dict
for key in model_list[0].state_dict():
    print(key)

module.RCNN_rpn.RPN_Conv.weight
module.RCNN_rpn.RPN_Conv.bias
module.RCNN_rpn.RPN_cls_score.weight
module.RCNN_rpn.RPN_cls_score.bias
module.RCNN_rpn.RPN_bbox_pred.weight
module.RCNN_rpn.RPN_bbox_pred.bias
module.RCNN_base.0.weight
module.RCNN_base.0.bias
module.RCNN_base.2.weight
module.RCNN_base.2.bias
module.RCNN_base.5.weight
module.RCNN_base.5.bias
module.RCNN_base.7.weight
module.RCNN_base.7.bias
module.RCNN_base.10.weight
module.RCNN_base.10.bias
module.RCNN_base.12.weight
module.RCNN_base.12.bias
module.RCNN_base.14.weight
module.RCNN_base.14.bias
module.RCNN_base.17.weight
module.RCNN_base.17.bias
module.RCNN_base.19.weight
module.RCNN_base.19.bias
module.RCNN_base.21.weight
module.RCNN_base.21.bias
module.RCNN_base.24.weight
module.RCNN_base.24.bias
module.RCNN_base.26.weight
module.RCNN_base.26.bias
module.RCNN_base.28.weight
module.RCNN_base.28.bias
module.RCNN_top.0.weight
module.RCNN_top.0.bias
module.RCNN_top.3.weight
module.RCNN_top.3.bias
module.RCNN_cls_score.weight
m

In [9]:
model_list[0].module

vgg16(
  (RCNN_rpn): _RPN(
    (RPN_Conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (RPN_cls_score): Conv2d(512, 24, kernel_size=(1, 1), stride=(1, 1))
    (RPN_bbox_pred): Conv2d(512, 48, kernel_size=(1, 1), stride=(1, 1))
    (RPN_proposal): _ProposalLayer()
    (RPN_anchor_target): _AnchorTargetLayer()
  )
  (RCNN_proposal_target): _ProposalTargetLayer()
  (RCNN_roi_pool): ROIPool(output_size=(7, 7), spatial_scale=0.0625)
  (RCNN_roi_align): ROIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0)
  (RCNN_base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), 

In [22]:
model_list[0].module.RCNN_base

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [16]:
for l in list(model_list[0].module.RCNN_base.modules())[0]:
    print(l)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(512, 512, kernel_size=(3, 3), stride

In [34]:
ratio_list = [0.33333,0.33333,0.33333]   
model_tmp=[None] * parties


for idx, my_model in enumerate(model_list):
    model_tmp[idx] = my_model.module.RCNN_base.state_dict()
    
for key in model_tmp[0]:  
    print('key={}'.format(key))
    model_avg = 0
    for idx, model_tmp_content in enumerate(model_tmp):     # add each model              
        #print(model_tmp_content)
        model_avg += ratio_list[idx] * model_tmp_content[key]
            
        #for i in range(len(model_tmp)):  #copy to each model            
        #    model_tmp[i][key] = model_avg

key=0.weight
key=0.bias
key=2.weight
key=2.bias
key=5.weight
key=5.bias
key=7.weight
key=7.bias
key=10.weight
key=10.bias
key=12.weight
key=12.bias
key=14.weight
key=14.bias
key=17.weight
key=17.bias
key=19.weight
key=19.bias
key=21.weight
key=21.bias
key=24.weight
key=24.bias
key=26.weight
key=26.bias
key=28.weight
key=28.bias


In [52]:
for model in model_list:
    count=0
    for key in model.module.RCNN_base.state_dict():
        print(key)
        print(model.module.RCNN_base.state_dict()[key])
        if count>2:
            break
        count+=1
    

0.weight
tensor([[[[ 4.2947e-01,  3.7346e-01, -6.1359e-02],
          [ 2.7477e-01,  3.8680e-02, -3.6722e-01],
          [-5.7468e-02, -2.6225e-01, -3.5009e-01]],

         [[ 5.5037e-01,  4.4007e-01, -8.1386e-02],
          [ 3.4574e-01,  4.0632e-02, -4.5350e-01],
          [-5.8634e-02, -3.3067e-01, -4.8503e-01]],

         [[ 4.8001e-01,  4.0854e-01, -6.5145e-02],
          [ 3.1047e-01,  5.0202e-02, -4.0338e-01],
          [-5.0871e-02, -2.8522e-01, -4.1851e-01]]],


        [[[ 1.1727e-01,  1.6206e-01,  1.3569e-01],
          [ 1.4835e-01,  2.0230e-01,  1.6169e-01],
          [ 1.2934e-01,  1.7157e-01,  1.3871e-01]],

         [[ 2.0877e-02,  4.7341e-02,  4.1854e-02],
          [ 3.1049e-02,  6.5810e-02,  4.6257e-02],
          [ 3.1678e-02,  5.4710e-02,  4.2319e-02]],

         [[-1.7269e-01, -1.7037e-01, -1.5435e-01],
          [-1.8760e-01, -1.7757e-01, -1.7440e-01],
          [-1.6600e-01, -1.6667e-01, -1.5705e-01]]],


        [[[ 3.4013e-02,  1.7086e-03, -1.1569e-01],
      

tensor([[[[ 1.6622e-01,  2.1201e-01, -5.9169e-03],
          [ 1.9920e-01, -2.5941e-02, -9.8491e-02],
          [-1.1158e-02, -1.0211e-01,  2.7340e-02]],

         [[ 9.4694e-03, -2.6217e-03, -3.9376e-03],
          [-1.3866e-03, -4.7829e-03,  4.7053e-03],
          [-6.1458e-03,  2.6535e-03,  9.4837e-03]],

         [[ 6.3268e-02,  2.2720e-02, -3.0701e-02],
          [ 2.1426e-02, -3.8117e-02, -2.2889e-02],
          [-1.0011e-02, -1.5070e-02,  3.7349e-02]],

         ...,

         [[ 3.4749e-03, -2.7030e-03,  5.8333e-03],
          [-1.4780e-02, -1.7993e-02,  2.9327e-05],
          [-1.6886e-02, -5.4378e-03, -2.6041e-02]],

         [[ 5.9723e-02, -3.2050e-02, -7.8093e-02],
          [-1.3928e-02, -8.9153e-02, -4.5611e-03],
          [-1.3414e-02, -2.2189e-02,  2.6749e-02]],

         [[ 1.3346e-01,  1.6811e-02, -7.8263e-02],
          [ 2.5567e-02, -1.0002e-01, -3.7962e-02],
          [-2.8886e-02, -2.6116e-02,  7.9885e-02]]],


        [[[ 1.4270e-01,  2.1013e-02, -2.2622e-02],
  